## Author : Daniel Anietie Eneh 
## Date : 20th June 2024

## Implementation of Yolov8 Architecture Under the Hood (From Scratch)

In this Notebook I begin by examining the Yolov8 Object Detection Model and the Yolov8 segmentation model respectively, then I attempt to build a similar architecture of both of them from scratch

![Alt Text](yolov8.png)


[Photo Reference_Link](https://arxiv.org/html/2304.00501v6)


In [1]:
!pip install ultralytics==8.0.196
# Necessary libraries
from ultralytics import YOLO
import torch.nn as nn
import math
import os
import cv2


In [2]:
import numpy
import torch

### 1. Xraying the Yolov8 Object Detection Model

This would enable us build our own custom architecture from scratch

In [3]:
#For Yolo Object Detection Model
model_dict = {'nano':'yolov8n.pt','small':'yolov8s.pt','medium':'yolov8m.pt', 'large':'yolov8l.pt'}

for model_name, model_weight in model_dict.items() :
    #print (f"{model_name} : {model_weight}"
    model = YOLO(model_weight)
    model_parameters = model.parameters()
    print(f"YOLOV-{model_name}: {sum(p.numel() for p in model_parameters)/1e6} million parameters")
    print(model.model)
    print ("__________________________________________________________________________________________________________________________________________________________________________")
    print ("")
        

/home/daniel/miniconda3/lib/python3.11/site-packages/ultralytics/nn/tasks.py:567: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(file, map_location='cpu'), 

YOLOV-nano: 3.1572 million parameters
DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C2f(
      (cv1): Conv(
        (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        

### 2. Xraying the Yolov8 Segmentation Model

This would enable us build our own custom architecture from scratch

In [4]:
#For Object Detection Yolo
#model_dict = {'nano':'yolov8n.pt','small':'yolov8s.pt','medium':'yolov8m.pt', 'large':'yolov8l.pt'}

#For Yolo Segmentation Model
model_dict = {'nano':'yolov8n-seg.pt','small':'yolov8s-seg.pt','medium':'yolov8m-seg.pt', 'large':'yolov8l-seg.pt'}


for model_name, model_weight in model_dict.items() :
    #print (f"{model_name} : {model_weight}"
    model = YOLO(model_weight)
    model_parameters = model.parameters()
    print(f"YOLOV-{model_name}: {sum(p.numel() for p in model_parameters)/1e6} million parameters")
    print(model.model)
    print ("__________________________________________________________________________________________________________________________________________________________________________")
    print ("")
        

YOLOV-nano: 3.409968 million parameters
SegmentationModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C2f(
      (cv1): Conv(
        (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
   

### The Backbone architecture
From the digaram architecture the back bone comprises of blocks Conv, C2f, SPPF.
1. Conv --> Conv2d + BatchNorm2d + SILU
2. C2f (cross-stage partial bottleneck with 2 convolutions) : Conv + Bottlenecks + Conv :
   This combines high-level features with contextual information to improve detection accuracy.

3. SPFF (Spatial pyramid pooling fast) : Conv + Maxpool2d + Conv
   Process features at various scales and pool them into a fixed-size feature map
   
    

(a) Conv

In [5]:
class Conv(nn.Module):
    def __init__(self,in_channels,out_channels, kernel_size=3,stride=1,padding=1,groups=1,activation=True):
        super().__init__() # Calls the constructor of the parent class
        self.conv=nn.Conv2d(in_channels, out_channels,kernel_size,stride,padding, bias = False, groups=groups)
        self.bn=nn.BatchNorm2d(out_channels,eps=0.001,momentum=0.03)
        self.act= nn.SiLU(inplace=True) if activation else nn.Identity()

    def forward(self,x):
        return self.act(self.bn(self.conv(x)))

b) C2f

In [6]:
#Bottle neck: Stack of 2 conv with short cutd connection (True/False)
class Bottleneck(nn.Module):
    def __init__(self, in_channels,out_channels,shortcut=True):
        super().__init__()
        self.conv1=Conv(in_channels, out_channels,kernel_size=3,stride=1, padding=1)
        self.conv2=Conv(out_channels,out_channels,kernel_size=3, stride=1,padding=1)
        self.shortcut=shortcut
        
    def forward(self,x):
        x_in = x # for residual connection
        x = self.conv1(x)
        x = self.conv2(x)
        if self.shortcut:
            x = x + x_in
        return x
#2
class C2f (nn.Module):
    def __init__(self, in_channels, out_channels, num_bottlenecks, shortcut=True):
        super().__init__()
        self.mid_channels = out_channels // 2
        self.num_bottlenecks = num_bottlenecks
        self.conv1 = Conv(in_channels, out_channels,kernel_size=1,stride=1,padding=0)

        #Sequence of bottleneck layers
        self.m=nn.ModuleList ([Bottleneck(self.mid_channels,self.mid_channels) for _ in range(num_bottlenecks)])
        self.conv2 = Conv((num_bottlenecks+2) * out_channels//2, out_channels, kernel_size=1, stride=1, padding=0)

    def forward (self, x):
        x = self.conv1(x)
        # split x along channel dimension
        x1,x2=x[:,:x.shape[1]//2,:,:], x[:,x.shape[1]//2:,:,:]

        # list of outputs
        outputs = [x1,x2] #x1 fed through the bottlenecks

        for i in range(self.num_bottlenecks):
            x1 = self.m[i](x1)
            outputs.insert(0,x1)
        outputs = torch.cat(outputs,dim=1)    
        out = self.conv2(outputs)

        return out
c2f = C2f (in_channels=64, out_channels=128, num_bottlenecks=2)
print(f"{sum(p.numel() for p in c2f.parameters())/1e6} million parameters")
dummy_input=torch.rand((1,64,244,244))
dummy_output=c2f(dummy_input)
print ("Output Shape: ", dummy_output.shape)

0.18944 million parameters
Output Shape:  torch.Size([1, 128, 244, 244])


c) SPPF

In [7]:
class SPPF(nn.Module):
    def __init__(self, in_channels, out_channels,kernel_size=5):
        #Kernel_size = size of maxpool
        super().__init__()
        hidden_channels = in_channels//2
        self.conv1=Conv(in_channels,hidden_channels,kernel_size=1,stride=1,padding=0)
        #concanete output
        self.conv2 = Conv(4 * hidden_channels, out_channels, kernel_size=1, stride=1, padding=0)

        # maxpool been applied to 3 different scales
        self.m = nn.MaxPool2d(kernel_size=kernel_size, stride=1, padding=kernel_size//2, dilation=1, ceil_mode=False)

    def forward(self,x):
        x = self.conv1(x)
        #apply maxpooling at 3 different scales
        y1 = self.m(x)
        y2 = self.m(y1)
        y3 = self.m(y2)

        # concatenate
        y = torch.cat([x,y1,y2,y3], dim=1)

        # final conv
        y = self.conv2(y)

        return y

# Check
sppf = SPPF(in_channels=128, out_channels=512)
print (f"{sum(p.numel() for p in sppf.parameters())/1e6} million parameters")

dummy_input = torch.rand((1,128,244,244))
dummy_output=sppf(dummy_input)
print ("Output Shape: ", dummy_output.shape)
        

0.140416 million parameters
Output Shape:  torch.Size([1, 512, 244, 244])


### Organizing the Backbone

In [8]:

def yolo_params(version):
    if version=='n':
        return 1/3,1/4,2.0
    elif version=='s':
        return 1/3, 1/2,2.0
    elif version=='m':
        return 2/3,3/4,1.5
    elif version=='l':
        return 1.0,1.0,1.0
    elif version=='x':
        return 1.0,1.25,1.0

class Backbone (nn.Module):
    def __init__(self,version,in_channels=3, shortcut=True):
        super().__init__()
        d,w,r = yolo_params(version)

        #Convolution layers
        self.conv_0= Conv(in_channels,int(64*w), kernel_size=3, stride=2, padding=1)
        self.conv_1=Conv(int(64*w), int(128*w), kernel_size=3, stride=2, padding=1)
        self.conv_3=Conv(int(128*w), int(256*w),kernel_size=3,stride=2,padding=1)
        self.conv_5=Conv(int(256*w),int(512*w),kernel_size=3,stride=2,padding=1)
        self.conv_7=Conv(int(512*w),int(512*w*r),kernel_size=3,stride=2,padding=1)

        #C2f layers
        self.c2f_2=C2f(int(128*w),int(128*w), num_bottlenecks=int(3*d),shortcut=True)
        self.c2f_4=C2f(int(256*w),int(256*w),num_bottlenecks=int(6*d),shortcut=True)
        self.c2f_6=C2f(int(512*w),int(512*w), num_bottlenecks=int(6*d),shortcut=True)
        self.c2f_8=C2f(int(512*w*r), int(512*w*r), num_bottlenecks=int(3*d),shortcut=True)

        #SPPF
        self.sppf=SPPF(int(512*w*r), int(512*w*r))

    def forward(self,x):
        x = self.conv_0(x)
        x = self.conv_1(x)
        x = self.conv_3(x)
        out1 = self.c2f_4(x) # Keep fpr Output
        x = self.conv_5(out1)

        out2 = self.c2f_6(x) #keep for output
        x = self.conv_7(out2)
        x = self.c2f_8(x)
        out3 = self.sppf(x)

        return out1, out2, out3

print("--------Nano Model--------")
backbone_n = Backbone(version='n')
print(f'{sum(p.numel() for p in backbone_n.parameters())/1e6} million parameters')

print("--------Small Model--------")
backbone_n = Backbone(version='s')
print(f'{sum(p.numel() for p in backbone_n.parameters())/1e6} million parameters')

print("--------Medium Model--------")
backbone_n = Backbone(version='m')
print(f'{sum(p.numel() for p in backbone_n.parameters())/1e6} million parameters')

print("--------Large Model--------")
backbone_n = Backbone(version='l')
print(f'{sum(p.numel() for p in backbone_n.parameters())/1e6} million parameters')


--------Nano Model--------
1.272656 million parameters
--------Small Model--------
5.079712 million parameters
--------Medium Model--------
11.855856 million parameters
--------Large Model--------
19.808576 million parameters


In [9]:
#check
backbone_n = Backbone(version='n')
x = torch.rand(1,3,640,640)
out1,out2,out3 = backbone_n(x)
print (out1.shape)
print (out2.shape)
print (out3.shape)

torch.Size([1, 64, 80, 80])
torch.Size([1, 128, 40, 40])
torch.Size([1, 256, 20, 20])


### Neck Architecture
The Neck consist of Upsample + C2f with <br>
**Upsample** = nearest-neighbour interpolation with scale factor=2. It does not have trainable parameters.

In [10]:
#upsample = nearest - neighbour interpolation with scale_factor = 2 does not have trainable parameters
class Upsample (nn.Module):
    def __init__(self, scale_factor=2,mode="nearest"):
        super().__init__()
        self.scale_factors=scale_factor
        self.mode = mode

    def forward (self,x):
        return nn.functional.interpolate(x,scale_factor=self.scale_factors,mode=self.mode)

In [11]:
class Neck (nn.Module):
    def __init__(self,version):
        super().__init__()
        d,w,r = yolo_params(version)
        self.up = Upsample() #no trainable parameters
        self.c2f_1=C2f(in_channels=int(512*w*(1+r)), out_channels=int(512*w),num_bottlenecks=int(3*d),shortcut=False)
        self.c2f_2=C2f(in_channels=int(768*w), out_channels=int(256*w),num_bottlenecks=int(3*d),shortcut=False)
        self.c2f_3=C2f(in_channels=int(768*w), out_channels=int(512*w),num_bottlenecks=int(3*d),shortcut=False)
        self.c2f_4=C2f(in_channels=int(512*w*(1+r)), out_channels=int(512*w*r),num_bottlenecks=int(3*d),shortcut=False)

        self.cv_1 = Conv(in_channels=int(256*w),out_channels=int(256*w),kernel_size=3,stride =2, padding=1)
        self.cv_2 = Conv(in_channels=int(512*w),out_channels=int(512*w),kernel_size=3,stride=2, padding=1)

    def forward (self, x_res_1, x_res_2,x):
        # x_res_1, _res_2 = output of backbone
        res_1 = x # for residual connection
        x = self.up(x)
        x = torch.cat([x,x_res_2],dim=1)
        res_2 = self.c2f_1(x) #for residual connection
        x = self.up(res_2)
        x = torch.cat([x,x_res_1],dim=1)

        out_1 = self.c2f_2(x)
        x=self.cv_1(out_1)
        x=torch.cat([x,res_2],dim=1)
        out_2 = self.c2f_3(x)
        x = self.cv_2(out_2)
        x = torch.cat([x,res_1],dim=1)
        out_3 = self.c2f_4(x)
        return out_1, out_2, out_3

#check
neck = Neck(version='n')
print(f'{sum(p.numel() for p in neck.parameters())/1e6} million parameters')
x = torch.rand((1,3,640,640))
out1,out2,out3 = Backbone(version='n')(x)
out_1, out_2, out_3 = neck(out1,out2,out3)
print(out_1.shape)
print (out_2.shape)
print (out_3.shape)
        
        

0.98688 million parameters
torch.Size([1, 64, 80, 80])
torch.Size([1, 128, 40, 40])
torch.Size([1, 256, 20, 20])


### Head Architecture
The head consist of 3 modules 1) bbox coordinates 2) classification scores 3) Distribution Focal loss (DFL) <br>

DFL considers the predicted bbox cordinates as a probability distribution. At inference time, it samples from the distribution to get **refined coordinates** (x,y,w,h). <br>
For example to predict coordinate x in the normalized ranger [0 1]

1. DFL uses 16 bins which are equally space in [0,1], bin length = 1/16
2. The model outputs 16 numbers which corresponds to probabilities that fall for e.g [0,0,...9/10,1/10]
3. Prediction for x = mean value = 9 / 10 . 15/16 + 1/10 .1 = 0,94375

#### a) DF

In [18]:
#DFL 
class DFL(nn.Module):
    def __init__(self,ch=16):
        super().__init__()
        self.ch = ch
        self.conv=nn.Conv2d(in_channels=ch,out_channels=1, kernel_size=1,bias=False).requires_grad_(False)

        #Initialize copy with (0, ...., ch-1)
        x = torch.arange(ch,dtype=torch.float).view(1,ch,1,1)
        self.conv.weight.data[:] = torch.nn.Parameter(x) # DFL only has ch parameters
    def forward(self,x):
        # x must have num_channels = 4 * ch: x = [bs, 4 * ch, c]
        b,c,a = x.shape
        x = x.view(b,4,self.ch,a).transpose(1,2)

        #Softmax on channel dimension to get distribution probavilities
        x=x.softmax(1)    #[b,ch,4.a]
        x=self.conv(x)    #[b,1,4,a]
        return x.view(b,4,a) #[b,4,a]

dummy_input=torch.rand((1,64,128))
dfl = DFL()
print (f"{sum(p.numel() for p in dfl.parameters())} parameters")
dummy_output=dfl(dummy_input)
print(dummy_output.shape)
print(dfl)

16 parameters
torch.Size([1, 4, 128])
DFL(
  (conv): Conv2d(16, 1, kernel_size=(1, 1), stride=(1, 1), bias=False)
)


### b) Head

In [26]:
class Head(nn.Module):
    def __init__(self,version,ch=16,num_classes=80):

        super().__init__()
        self.ch=ch                          # dfl channels
        self.coordinates=self.ch*4          # number of bounding box coordinates 
        self.nc=num_classes                 # 80 for COCO
        self.no=self.coordinates+self.nc    # number of outputs per anchor box

        self.stride=torch.zeros(3)          # strides computed during build
        
        d,w,r=yolo_params(version=version)
        
        # for bounding boxes
        self.box=nn.ModuleList([
            nn.Sequential(Conv(int(256*w),self.coordinates,kernel_size=3,stride=1,padding=1),
                          Conv(self.coordinates,self.coordinates,kernel_size=3,stride=1,padding=1),
                          nn.Conv2d(self.coordinates,self.coordinates,kernel_size=1,stride=1)),

            nn.Sequential(Conv(int(512*w),self.coordinates,kernel_size=3,stride=1,padding=1),
                          Conv(self.coordinates,self.coordinates,kernel_size=3,stride=1,padding=1),
                          nn.Conv2d(self.coordinates,self.coordinates,kernel_size=1,stride=1)),

            nn.Sequential(Conv(int(512*w*r),self.coordinates,kernel_size=3,stride=1,padding=1),
                          Conv(self.coordinates,self.coordinates,kernel_size=3,stride=1,padding=1),
                          nn.Conv2d(self.coordinates,self.coordinates,kernel_size=1,stride=1))
        ])

        # for classification
        self.cls=nn.ModuleList([
            nn.Sequential(Conv(int(256*w),self.nc,kernel_size=3,stride=1,padding=1),
                          Conv(self.nc,self.nc,kernel_size=3,stride=1,padding=1),
                          nn.Conv2d(self.nc,self.nc,kernel_size=1,stride=1)),

            nn.Sequential(Conv(int(512*w),self.nc,kernel_size=3,stride=1,padding=1),
                          Conv(self.nc,self.nc,kernel_size=3,stride=1,padding=1),
                          nn.Conv2d(self.nc,self.nc,kernel_size=1,stride=1)),

            nn.Sequential(Conv(int(512*w*r),self.nc,kernel_size=3,stride=1,padding=1),
                          Conv(self.nc,self.nc,kernel_size=3,stride=1,padding=1),
                          nn.Conv2d(self.nc,self.nc,kernel_size=1,stride=1))
        ])

        # dfl
        self.dfl=DFL()

    def forward(self,x):
        # x = output of Neck = list of 3 tensors with different resolution and different channel dim
        #     x[0]=[bs, ch0, w0, h0], x[1]=[bs, ch1, w1, h1], x[2]=[bs,ch2, w2, h2] 

        for i in range(len(self.box)):       # detection head i
            box=self.box[i](x[i])            # [bs,num_coordinates,w,h]
            cls=self.cls[i](x[i])            # [bs,num_classes,w,h]
            x[i]=torch.cat((box,cls),dim=1)  # [bs,num_coordinates+num_classes,w,h]

        # in training, no dfl output
        if self.training:
            return x                         # [3,bs,num_coordinates+num_classes,w,h]
        
        # in inference time, dfl produces refined bounding box coordinates
        anchors, strides = (i.transpose(0, 1) for i in self.make_anchors(x, self.stride))

        # concatenate predictions from all detection layers
        x = torch.cat([i.view(x[0].shape[0], self.no, -1) for i in x], dim=2) #[bs, 4*self.ch + self.nc, sum_i(h[i]w[i])]
        
        # split out predictions for box and cls
        #           box=[bs,4×self.ch,sum_i(h[i]w[i])]
        #           cls=[bs,self.nc,sum_i(h[i]w[i])]
        box, cls = x.split(split_size=(4 * self.ch, self.nc), dim=1)


        a, b = self.dfl(box).chunk(2, 1)  # a=b=[bs,2×self.ch,sum_i(h[i]w[i])]
        a = anchors.unsqueeze(0) - a
        b = anchors.unsqueeze(0) + b
        box = torch.cat(tensors=((a + b) / 2, b - a), dim=1)
        
        return torch.cat(tensors=(box * strides, cls.sigmoid()), dim=1)


    def make_anchors(self, x, strides, offset=0.5):
        # x= list of feature maps: x=[x[0],...,x[N-1]], in our case N= num_detection_heads=3
        #                          each having shape [bs,ch,w,h]
        #    each feature map x[i] gives output[i] = w*h anchor coordinates + w*h stride values
        
        # strides = list of stride values indicating how much 
        #           the spatial resolution of the feature map is reduced compared to the original image

        assert x is not None
        anchor_tensor, stride_tensor = [], []
        dtype, device = x[0].dtype, x[0].device
        for i, stride in enumerate(strides):
            _, _, h, w = x[i].shape
            sx = torch.arange(end=w, device=device, dtype=dtype) + offset  # x coordinates of anchor centers
            sy = torch.arange(end=h, device=device, dtype=dtype) + offset  # y coordinates of anchor centers
            sy, sx = torch.meshgrid(sy, sx)                                # all anchor centers 
            anchor_tensor.append(torch.stack((sx, sy), -1).view(-1, 2))
            stride_tensor.append(torch.full((h * w, 1), stride, dtype=dtype, device=device))
        return torch.cat(anchor_tensor), torch.cat(stride_tensor)


### Organizing Everything together

In [27]:
detect=Head(version='n')
print (f"{sum(p.numel() for p in detect.parameters())/1e6} million parameters")

#out_1,out_2, out_3, are output of the neck

output = detect([out_1, out_2,out_3])
print (output[0].shape)
print (output[1].shape)
print (output[2].shape)
print (detect)

0.897664 million parameters
torch.Size([1, 144, 80, 80])
torch.Size([1, 144, 40, 40])
torch.Size([1, 144, 20, 20])
Head(
  (box): ModuleList(
    (0): Sequential(
      (0): Conv(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
    )
    (1): Sequential(
      (0): Conv(
        (conv): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
    

## Creating My Unique Model

In [30]:
class my_model(nn.Module):
    def __init__(self,version):
        super().__init__()
        self.backbone=Backbone(version=version)
        self.neck= Neck(version=version)
        self.head=Head(version=version)
    def forward(self,x):
        x = self.backbone(x)
        x = self.neck(x[0],x[1],x[2])
        return self.head(list(x))
model = my_model(version='n')
print (f"{sum(p.numel() for p in model.parameters())/1e6} million parameters")
print (model)

3.1572 million parameters
my_model(
  (backbone): Backbone(
    (conv_0): Conv(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (conv_1): Conv(
      (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (conv_3): Conv(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (conv_5): Conv(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
     

### Training with on a Dataset with My custom Model


In [35]:
!pip install utils
import yaml
from utils.dataset import *
from touch.utils import data

#get all file names
data_dir ='COCO'
filenames_train = []
with open(f'{data_dir}/train2017.txt') as reader:
    for filename in reader.readlines():
        filename = os.path.basename(filename.rstrip())
        file_names_train.append(f'{data_dir}/images/train2017/' + filename)

input_size=640

with open ('utils/args.yaml', errors='ignore') as f:
    params = yaml.safe_load(f)

train_data=Dataset(filenames_train,input_size,params,argument=True)
train_loader=data.DataLoader(train_data,batch_size=64,num_workers=0, pin_memory=True, collate_fn=Dataset.collate_fn)
print(f"Train loader : {len(train_loader)} batches")
    



ModuleNotFoundError: No module named 'utils.dataset'